In [1]:
import os

In [2]:
os.chdir("../")
%pwd

'c:\\Users\\sridhar\\Desktop\\DATA ANALYTICS PROJECTS\\DATA SCIENCE\\End-To-End-ML-Project'

In [3]:
from dataclasses import dataclass  # Importing the dataclass decorator.
from pathlib import Path  # Object-oriented interface to filesystem paths.

@dataclass(frozen=True)
class ModelEvaluationConfig:
    """
    Data class for configuration related to model evaluation.

    Attributes:
        root_dir (Path): Root directory for model evaluation artifacts.
        test_data_path (Path): Path to the test data CSV file.
        model_path (Path): Path to the trained machine learning model.
        all_params (dict): Dictionary containing all relevant parameters for model evaluation.
        metric_file_name (Path): File name for saving model evaluation metrics.
        target_column (str): Name of the target column in the dataset.
    """
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str


In [7]:
from mlproject.constants import *
from mlproject.utils.common import read_yaml,create_directories,save_json

In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH):
        """
        Initializes the ConfigurationManager with default or provided file paths.

        Args:
            config_filepath (Path, optional): Path to the configuration file. Defaults to CONFIG_FILE_PATH.
            params_filepath (Path, optional): Path to the parameters file. Defaults to PARAMS_FILE_PATH.
            schema_filepath (Path, optional): Path to the schema file. Defaults to SCHEMA_FILE_PATH.
        """
        # Reading configuration, parameters, and schema files using read_yaml function.
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        # Creating the root directory for project artifacts.
        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        """
        Retrieves the configuration for model evaluation.

        Returns:
            ModelEvaluationConfig: Data class containing model evaluation configuration.
        """
        # Extracting model evaluation configuration, ElasticNet parameters, and target column from the overall project configuration.
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        # Creating the root directory for model evaluation artifacts.
        create_directories([config.root_dir])

        # Creating a ModelEvaluationConfig object with the extracted configuration.
        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name
        )

        return model_evaluation_config


In [6]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import numpy as np
import joblib

In [8]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        """
        Initializes the ModelEvaluation object with the provided configuration.

        Args:
            config (ModelEvaluationConfig): ModelEvaluationConfig object containing model evaluation settings.
        """
        self.config = config

    def eval_metrics(self, actual, pred):
        """
        Calculates evaluation metrics including RMSE, MAE, and R2.

        Args:
            actual: Ground truth values.
            pred: Predicted values.

        Returns:
            Tuple: Tuple containing RMSE, MAE, and R2.
        """
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2

    def save_results(self):
        """
        Loads the test data and the trained model, predicts on the test data, calculates metrics, and saves them.

        Returns:
            None
        """
        # Reading test data and trained model.
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        # Extracting features and target variable from the test data.
        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        # Predicting target variable on the test data.
        predicted_qualities = model.predict(test_x)

        # Calculating evaluation metrics.
        (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)

        # Saving metrics as a JSON file locally.
        scores = {"rmse": rmse, "mae": mae, "r2": r2}
        save_json(path=Path(self.config.metric_file_name), data=scores)


In [9]:
try:
    # Creating ConfigurationManager to manage project configurations.
    config = ConfigurationManager()

    # Retrieving model evaluation configuration.
    model_evaluation_config = config.get_model_evaluation_config()

    # Creating ModelEvaluation object with the obtained configuration.
    model_evaluation = ModelEvaluation(config=model_evaluation_config)

    # Saving evaluation results, including metrics.
    model_evaluation.save_results()

except Exception as e:
    # Handling and re-raising any exceptions that occur during the process.
    raise e


[2023-12-02 23:03:12,039: INFO: common: YAML file: config\config.yaml loaded successfully]
[2023-12-02 23:03:12,047: INFO: common: YAML file: params.yaml loaded successfully]
[2023-12-02 23:03:12,054: INFO: common: YAML file: schema.yaml loaded successfully]
[2023-12-02 23:03:12,056: INFO: common: Created directory at: artifacts]
[2023-12-02 23:03:12,057: INFO: common: Created directory at: artifacts/model_evaluation]
[2023-12-02 23:03:12,218: INFO: common: JSON file saved at: artifacts\model_evaluation\metrics.json]
